In [21]:
import pandas as pd
import numpy as np
from whoswho import who
from rapidfuzz import process

In [11]:
def stringToList(x):
    return x.strip('][\"\'').split(', ') if not pd.isnull(x) else x

In [12]:
matching = pd.read_csv("Data/final_matching.csv", index_col = 0)
matching_post1790 = pd.read_csv("Data/final_matching_post1790.csv", index_col = 0)

In [13]:
prepost_match = pd.read_csv("prepost_matched_debt_files.csv", index_col = 0)
prepost_match['full name'] = prepost_match['full name'].apply(lambda x: stringToList(x))
prepost_match['Same State Loan Office'] = prepost_match['Same State Loan Office'].apply(lambda x: 
                                                                  stringToList(x))
prepost_match['Different State Loan Office'] = prepost_match['Different State Loan Office'].apply(lambda x: 
                                                                            stringToList(x))
prepost_match['Same State Liquidated Debt'] = prepost_match['Same State Liquidated Debt'].apply(lambda x: 
                                                                          stringToList(x))
prepost_match['Different State Liquidated Debt'] = prepost_match['Different State Liquidated Debt'].apply(lambda x: 
                                                                                    stringToList(x))
prepost_match['Pierce Certificates'] = prepost_match['Pierce Certificates'].apply(lambda x: 
                                                            stringToList(x))

In [29]:
temp = prepost_match['full name'].apply(pd.Series)
temp[temp[1].apply(lambda x: not pd.isnull(x))]

,0,1,2
71,Abraham Cole','James Cole,NaN
151,Alexander Mowatt','Michael Mowatt,NaN
169,Allexander Gillon','Robert Smith,NaN
174,Amasa Keyes','Elnathan Keyes,NaN
197,Amshong','Barnwall,NaN
...,...,...,...
4667,William Constable','James Constable,NaN
4768,William Joseph','Richard Hart,NaN
4809,William Morris','Swanwick,NaN
4860,William Sherer','McDowell,NaN


In [14]:
colnames = ['full name', 'match']
def genMatches(colname):
    series_df = prepost_match[colname].apply(pd.Series)
    index_df = prepost_match['full name'].apply(pd.Series)
    df = pd.DataFrame(columns = colnames)
    for i in np.arange(len(series_df.columns)):
        for j in np.arange(len(index_df.columns)):
            df_temp = pd.concat([index_df[[j]], series_df[[i]]], axis = 1)
            df_temp.columns = colnames
            df_temp.dropna(inplace = True)
            df = pd.concat([df_temp, df]).drop_duplicates()
    df['full name'] = df['full name'].apply(lambda x: x.strip('\"\''))
    df['match'] = df['match'].apply(lambda x: x.strip('\"\''))
    return df

In [15]:
collist = ['Same State Loan Office', 'Different State Loan Office', 'Same State Liquidated Debt', 'Different State Liquidated Debt',
           'Pierce Certificates']
matches = pd.DataFrame(columns = colnames)
for column in collist:
    matches = pd.concat([matches, genMatches(column)], axis = 0).drop_duplicates()
matches = matches[matches['full name'] != matches['match']]    

In [17]:
matching_merged = pd.concat([matching, matching_post1790]).drop_duplicates()
matching_merged = matching_merged[matching_merged['Scores'] != 100]
matching_total = pd.concat([matching_merged, matches])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
matches

,full name,match
4865,William Smith,William D Smith
2908,James Wells,James Welles
2197,John Conklin,John Conkling
2908,Joshua Belden,James Welles
4048,Samuel Smith,John Smith
...,...,...
4718,William Graham,Wiliam Graham
4743,William Heth Treasurer SC,William Heth
4796,William Matson,William Mason
4812,William Moseley,William Mosely


In [20]:
matching_merged.columns = ['full name', 'match', 'Scores']
matches['Scores'] = [process.extract(fn, match) for fn, match in zip(matches['full name'], matches['match'])]

,full name,match
4865,William Smith,William D Smith
2908,James Wells,James Welles
2197,John Conklin,John Conkling
2908,Joshua Belden,James Welles
4048,Samuel Smith,John Smith
...,...,...
4718,William Graham,Wiliam Graham
4743,William Heth Treasurer SC,William Heth
4796,William Matson,William Mason
4812,William Moseley,William Mosely


In [19]:
matching_merged.columns = ['full name', 'match', 'Scores']

,Delegates,Loan Matches,Scores
27668,A. Fisher,John A Fisher,95.0
387,Abraham Baldwin,Abraham Baldwine,97.0
856,Allen McLane,Allen McClane,96.0
35167,Allen Samuel,Samuel Allen,95.0
43091,Allen Samuel,Samuel G Allen,95.0
...,...,...,...
14041,William Robinson,Williiam J Robinson,91.0
11387,William Robison,Williiam J Robinson,88.0
3376,William Smith,William Smith,96.0
11830,Williams John,John Williams,95.0
